In [13]:
import torch

import matplotlib.pyplot as plt
import numpy as np,pandas as pd

from sklearn.neighbors import KernelDensity
from scipy.optimize import minimize
from scipy.linalg import block_diag
from sklearn.covariance import LedoitWolf, MinCovDet
from deepdowmine.nn import BachelierNetWithShorting, BachelierNet, BachelierNetWithShortingUpd

import cvxpy as cp

In [8]:
def corr2cov(corr, std):
    cov = corr * np.outer(std, std)
    return cov  

In [14]:
#Generate a block-diagnoal covariance matrix and a vector of means
# Block represent sector
# bSize  number of assets in the sector
# bCorr correlation between assets in the sector
def formBlockMatrix(nBlocks, bSize, bCorr):
    block = np.ones( (bSize, bSize))*bCorr
    block[range(bSize), range(bSize)] = 1 #diagonal is 1
    corr = block_diag(*([block]*nBlocks))
    return corr



def formTrueMatrix(nBlocks, bSize, bCorr):
    corr0 = formBlockMatrix(nBlocks, bSize, bCorr)
    corr0 = pd.DataFrame(corr0)
    cols = corr0.columns.tolist()
    np.random.shuffle(cols)
    corr0 = corr0[cols].loc[cols].copy(deep=True)
    std0 = np.random.uniform(.05, .2, corr0.shape[0])
    cov0 = corr2cov(corr0, std0)
    mu0 = np.random.normal(std0, std0, cov0.shape[0]).reshape(-1,1)
    return mu0, cov0

def corr2cov(corr, std):
    cov = corr * np.outer(std, std)
    return cov

# Denoising of the empirical covariance matrix
# by constant residual eigenvalue method
def deNoiseCov(cov0,q,bWidth):
    corr0=cov2corr(cov0)
    eVal0,eVec0=getPCA(corr0)
    eMax0,var0=findMaxEval(np.diag(eVal0),q,bWidth)
    nFacts0=eVal0.shape[0]-np.diag(eVal0)[::-1].searchsorted(eMax0)
    corr1=denoisedCorr(eVal0,eVec0,nFacts0)
    cov1=corr2cov(corr1,np.diag(cov0)**.5)
    return cov1


# function to obtain empirical from true matrix with and without shrink
def simCovMu(mu0,cov0,nObs,shrink=False):
    x=np.random.multivariate_normal(mu0.flatten(),cov0,size=nObs)
    mu1=x.mean(axis=0).reshape(-1,1)
    if shrink:cov1=LedoitWolf().fit(x).covariance_
    else:cov1=np.cov(x,rowvar=0)
    return mu1,cov1, x


def simCovMuRob(mu0,cov0,nObs):
    x=np.random.multivariate_normal(mu0.flatten(),cov0,size=nObs)
    mu1=x.mean(axis=0).reshape(-1,1)
    cov1=MinCovDet().fit(x).covariance_
    return mu1,cov1, x


# replace in random eigenvalues by constants
def denoisedCorr(eVal, eVec, nFacts):
    eVal_ = np.diag(eVal).copy()
    eVal_[nFacts:] = eVal_[nFacts:].sum()/float(eVal_.shape[0] - nFacts) #all but 0..i values equals (1/N-i)sum(eVal_[i..N]))
    eVal_ = np.diag(eVal_) #square matrix with eigenvalues as diagonal: eVal_.I
    corr1 = np.dot(eVec, eVal_).dot(eVec.T) #Eigendecomposition of a symmetric matrix: S = QΛQT
    corr1 = cov2corr(corr1) # Rescaling the correlation matrix to have 1s on the main diagonal
    return corr1


def getRndCov(nCols, nFacts): #nFacts - contains signal out of nCols
    w = np.random.normal(size=(nCols, nFacts))
    cov = np.dot(w, w.T) #random cov matrix, however not full rank
    cov += np.diag(np.random.uniform(size=nCols)) #full rank cov
    return cov

def cov2corr(cov):
    # Derive the correlation matrix from a covariance matrix
    std = np.sqrt(np.diag(cov))
    corr = cov/np.outer(std,std)
    corr[corr<-1], corr[corr>1] = -1,1 #for numerical errors
    return corr
    
def corr2cov(corr, std):
    cov = corr * np.outer(std, std)
    return cov     
    
#snippet 2.4 - fitting the marcenko-pastur pdf - find variance
#Fit error
def errPDFs(var, eVal, q, bWidth, pts=1000):
    var = var[0]
    pdf0 = mpPDF(var, q, pts) #theoretical pdf
    pdf1 = fitKDE(eVal, bWidth, x=pdf0.index.values) #empirical pdf
    sse = np.sum((pdf1-pdf0)**2)
    print("sse:"+str(sse))
    return sse 
    
# find max random eVal by fitting Marcenko's dist
# and return variance
def findMaxEval(eVal, q, bWidth):
    out = minimize(lambda *x: errPDFs(*x), x0=np.array(0.5), args=(eVal, q, bWidth), bounds=((1E-5, 1-1E-5),))
    print("found errPDFs"+str(out['x'][0]))
    if out['success']: var = out['x'][0]
    else: var=1
    eMax = var*(1+(1./q)**.5)**2
    return eMax, var



def getPCA(matrix):
# Get eVal,eVec from a !!!Hermitian matrix (cov matrix is a hermitian matrix)
    eVal,eVec=np.linalg.eigh(matrix) 
    indices=eVal.argsort()[::-1] # arguments for sorting eVal desc
    eVal,eVec=eVal[indices],eVec[:,indices]
    eVal=np.diagflat(eVal)
    return eVal,eVec


def fitKDE(obs,bWidth=.25,kernel='gaussian',x=None):
    # Fit kernel to a series of obs, and derive the prob of obs
    # x is the array of values on which the fit KDE will be evaluated
    if len(obs.shape)==1:obs=obs.reshape(-1,1)
    kde=KernelDensity(kernel=kernel,bandwidth=bWidth).fit(obs)
    if x is None:x=np.unique(obs).reshape(-1,1)
    if len(x.shape)==1:x=x.reshape(-1,1)
    logProb=kde.score_samples(x) # log(density)
    pdf=pd.Series(np.exp(logProb),index=x.flatten())
    return pdf


def mpPDF(var,q,pts):
    # Marcenko-Pastur pdf
    # q=T/N (>1)
    # pts - amount of points
    eMin,eMax=var*(1-(1./q)**.5)**2,var*(1+(1./q)**.5)**2
    eVal=np.linspace(eMin,eMax,pts)
    pdf=q/(2*np.pi*var*eVal)*((eMax-eVal)*(eVal-eMin))**.5
    pdf=pd.Series(pdf,index=eVal)
    return pdf


def getRndCov(nCols, nFacts): #nFacts - contains signal out of nCols
    w = np.random.normal(size=(nCols, nFacts))
    cov = np.dot(w, w.T) #random cov matrix, however not full rank
    cov += np.diag(np.random.uniform(size=nCols)) #full rank cov
    return cov

def cov2corr(cov):
    # Derive the correlation matrix from a covariance matrix
    std = np.sqrt(np.diag(cov))
    corr = cov/np.outer(std,std)
    corr[corr<-1], corr[corr>1] = -1,1 #for numerical errors
    return corr
    
def corr2cov(corr, std):
    cov = corr * np.outer(std, std)
    return cov     
    
#snippet 2.4 - fitting the marcenko-pastur pdf - find variance
#Fit error
def errPDFs(var, eVal, q, bWidth, pts=1000):
    var = var[0]
    pdf0 = mpPDF(var, q, pts) #theoretical pdf
    pdf1 = fitKDE(eVal, bWidth, x=pdf0.index.values) #empirical pdf
    sse = np.sum((pdf1-pdf0)**2)
    print("sse:"+str(sse))
    return sse 
    
# find max random eVal by fitting Marcenko's dist
# and return variance
def findMaxEval(eVal, q, bWidth):
    out = minimize(lambda *x: errPDFs(*x), x0=np.array(0.5), args=(eVal, q, bWidth), bounds=((1E-5, 1-1E-5),))
    print("found errPDFs"+str(out['x'][0]))
    if out['success']: var = out['x'][0]
    else: var=1
    eMax = var*(1+(1./q)**.5)**2
    return eMax, var


# find min var portfolio
def optPort(cov,mu=None):
    inv=np.linalg.inv(cov)
    ones=np.ones(shape=(inv.shape[0],1))
    if mu is None:mu=ones
    w=np.dot(inv,mu)
    w/=np.dot(ones.T,w)
    return w




def minimize_portfolio_variance(cov_matrix):
    """
    Finds the portfolio weights that minimize the portfolio variance.
    
    :param cov_matrix: The covariance matrix of asset returns.
    :return: Optimal portfolio weights as a numpy array.
    """
    # Number of assets
    n = cov_matrix.shape[0]

    # Portfolio weights variables
    w = cp.Variable(n)

    # Portfolio variance
    port_variance = cp.quad_form(w, cov_matrix)

    # Objective Function: Minimize portfolio variance
    objective = cp.Minimize(port_variance)

    # Constraints: weights sum to 1, non-negativity
    constraints = [cp.sum(w) == 1]

    # Problem
    problem = cp.Problem(objective, constraints)
    
    # Solve the problem
    problem.solve()

    # Portfolio weights
    optimal_weights = w.value
    
    return optimal_weights


def transform_returns_to_Xy_tensors(returns, lookback, n_timesteps, horizon, gap):
    X_list, y_list = [], []

    for i in range(lookback, n_timesteps - horizon - gap + 1):
        X_list.append(returns[i - lookback: i, :])
        y_list.append(returns[i + gap: i + gap + horizon, :])

    X = np.stack(X_list, axis=0)[:, None, ...]
    y = np.stack(y_list, axis=0)[:, None, ...]
    
    return X, y

def X_to_tensor(X, loockback, i = None):
    
    
    if i:
        if i < loockback:
            raise ValueError("i must not be greater than lookback")
    else: i = -1
    # Parameters
    i = len(X) if i == -1 else i

    # Slicing X and reshaping
    X_slice = X[i - lookback: i, :]  # This is (40, 250)
    X_tensor = torch.tensor(X_slice).unsqueeze(0).unsqueeze(0)  # Adding two dimensions
    return torch.tensor(X_tensor, dtype=torch.float32)

In [10]:

lookback, gap, horizon = 40, 0, 20

In [11]:
# Should be run after the code below to obtain X!!!
n_timesteps, n_assets = X.shape
# tX, ty = transform_returns_to_Xy_tensors(X, lookback, n_timesteps, horizon, gap)
# res = torch.tensor(tX[[-1]], dtype=torch.float32)#(X[indices_train], dtype=torch.float32) #indices_train

NameError: name 'X' is not defined

# !!!!!!!!!!!!! Recheck how deep dow trains the data

In [ ]:
tensorX = X_to_tensor(X, 40)

In [ ]:
tensorX.shape

In [ ]:
X[[999], 0]

In [ ]:
tensorX[0, 0, 39, 0]

In [ ]:
X[[959, 960, 961], 0]

In [ ]:
tensorX[0, 0, 0, 0]

In [ ]:

arr = X[0:1000, 0]
value_to_find = 0.0678

# Since we are dealing with floating-point numbers, we use a small tolerance
tolerance = 1e-4

# Finding indices where value matches
indices = np.where(np.abs(arr - value_to_find) < tolerance)
indices

# !!!! End recheck

In [ ]:
network = network = BachelierNetWithShortingUpd(#BachelierNet(
        1,
        250,
        hidden_size=32,
        shrinkage_strategy="diagonal",
        p=0.5,
    )
#

# Experiment max sharpe

In [ ]:
# network.load_state_dict(torch.load('bachelier_with_shorting_250assets_max_sharpe_ratio_loss.pth'))


# np.random.seed(32)
# # block is a sector with bSize assets
# nBlocks, bSize, bCorr = 2, 125, .01 #5, 50, .5
# np.random.seed(0)
# mu0, cov0 = formTrueMatrix(nBlocks, bSize, bCorr)

# nObs, nTrials, bWidth, shrink = 1000, 15, .01, False
# w1 = pd.DataFrame(columns = range(cov0.shape[0]), index = range(nTrials), dtype=float)
# w1_d = w1.copy(deep=True)
# w1_n = w1.copy(deep=True)

# covs1 = []
# covs1_d = []
# covs1_n = []
# np.random.seed(0)
# for i in range(nTrials):
#     mu1, cov1, X = simCovMu(mu0, cov0, nObs, shrink = shrink)
#     cov1_d = deNoiseCov(cov1, nObs*1./cov1.shape[1], bWidth)
#     w1.loc[i] = optPort(cov1,mu1).flatten()   #optPort(cov1, mu1).flatten() # add column vector w as row in w1
#     w1_d.loc[i] = optPort(cov1_d,mu1).flatten() #optPort(cov1_d, mu1).flatten() # np.sum(w1_d, axis=1) is vector of 1's. sum(np.sum(w1_d, axis=0)= nTrials
#     # so minimum-variance-portfolio is 1./nTrials*(np.sum(w1_d, axis=0)) - but distribution not stationary
#     tensorX = X_to_tensor(X, lookback)
#     weights_n = network(tensorX)
#     w1_n.loc[i] = weights_n.detach().numpy().flatten()#network(tensorX)
    
    
#     covs1.append(cov1)
#     covs1_d.append(cov1_d)
    
# min_var_port = 1./nTrials*(np.sum(w1_d, axis=0)) 
# #code snippet 2.11
# w0 = optPort(cov0, mu0) # w0 true percentage asset allocation
# w0 = np.repeat(w0.T, w1.shape[0], axis=0) 
# rmsd = np.mean((w1-w0).values.flatten()**2)**.5     #RMSE not denoised
# rmsd_d = np.mean((w1_d-w0).values.flatten()**2)**.5 #RMSE denoised
# rmsd_n = np.mean((w1_n-w0).values.flatten()**2)**.5 #
# print("RMSE not denoised:"+str( rmsd))
# print("RMSE denoised:"+str( rmsd_d))
# print("RMSE neural:"+str( rmsd_n))

# Min var opt

## Not shrunk

In [17]:
# network.load_state_dict(torch.load('bachelier_with_shorting_250assets_min_var_loss.pth'))


np.random.seed(32)
# block is a sector with bSize assets
nBlocks, bSize, bCorr =10, 50, .5
np.random.seed(0)
mu0, cov0 = formTrueMatrix(nBlocks, bSize, bCorr)
Xs = []

nObs, nTrials, bWidth, shrink, minVarPortf = 10000, 15, .01, False, True
w1 = pd.DataFrame(columns = range(cov0.shape[0]), index = range(nTrials), dtype=float)
w1_d = w1.copy(deep=True)
# w1_n = w1.copy(deep=True)

covs1 = []
covs1_d = []
covs1_n = []
np.random.seed(0)
for i in range(nTrials):
    mu1, cov1, X = simCovMu(mu0, cov0, nObs, shrink = shrink)
    Xs.append(X)
    if minVarPortf: mu1 = None
    cov1_d = deNoiseCov(cov1, nObs*1./cov1.shape[1], bWidth)
    w1.loc[i] = minimize_portfolio_variance(cov1).flatten()   #optPort(cov1, mu1).flatten() # add column vector w as row in w1
    w1_d.loc[i] = minimize_portfolio_variance(cov1_d).flatten() #optPort(cov1_d, mu1).flatten() # np.sum(w1_d, axis=1) is vector of 1's. sum(np.sum(w1_d, axis=0)= nTrials
    # so minimum-variance-portfolio is 1./nTrials*(np.sum(w1_d, axis=0)) - but distribution not stationary
   # tensorX = X_to_tensor(X, lookback)
   # weights_n = network(tensorX)
   # w1_n.loc[i] = weights_n.detach().numpy().flatten()#network(tensorX)
    
    
    covs1.append(cov1)
    covs1_d.append(cov1_d)
    
min_var_port = 1./nTrials*(np.sum(w1_d, axis=0)) 
#code snippet 2.11
w0 = optPort(cov0, None if minVarPortf else mu0) # w0 true percentage asset allocation
w0 = np.repeat(w0.T, w1.shape[0], axis=0) 
rmsd = np.mean((w1-w0).values.flatten()**2)**.5     #RMSE not denoised
rmsd_d = np.mean((w1_d-w0).values.flatten()**2)**.5 #RMSE denoised
#rmsd_n = np.mean((w1_n-w0).values.flatten()**2)**.5 #
print("RMSE not denoised:"+str( rmsd))
print("RMSE denoised:"+str( rmsd_d))
#print("RMSE neural:"+str( rmsd_n))

sse:6.7788248961284
sse:6.778827138512861
sse:13774890325380.236
sse:13747381814369.688
sse:6744.109356768681
sse:6744.108357725492
sse:6.7437814620639775
sse:6.743780279929175
sse:6.730496772778256
sse:6.730496779171952
sse:6.730496385915422
sse:6.730496385933165
sse:6.730496385921468
sse:6.730496385921329
sse:6.730496385915857
sse:6.730496385931595
sse:6.730496385915527
sse:6.730496385933009
sse:6.730496385915535
sse:6.730496385933132
sse:6.730496385915447
sse:6.730496385933162
sse:6.730496385915422
sse:6.730496385933165
sse:6.730496385915501
sse:6.730496385933234
sse:6.730496385915422
sse:6.730496385933165
sse:6.730496385915534
sse:6.730496385933138
sse:6.730496385915422
sse:6.730496385933165
sse:6.730496385915476
sse:6.7304963859330975
sse:6.730496385915422
sse:6.730496385933165
sse:6.730496385915414
sse:6.730496385933115
sse:6.730496385915467
sse:6.730496385933176
sse:7.5667371621206385
sse:7.566727677011555
sse:6.730496385921224
sse:6.730496385921281
sse:6.730496385915916
sse:6.7

## Shrunk

In [16]:
# network.load_state_dict(torch.load('bachelier_with_shorting_250assets_min_var_loss.pth'))


np.random.seed(32)
# block is a sector with bSize assets
nBlocks, bSize, bCorr =10, 50, .5
np.random.seed(0)
mu0, cov0 = formTrueMatrix(nBlocks, bSize, bCorr)
Xs = []

nObs, nTrials, bWidth, shrink, minVarPortf = 10000, 15, .01, True, True
w1 = pd.DataFrame(columns = range(cov0.shape[0]), index = range(nTrials), dtype=float)
w1_d = w1.copy(deep=True)
# w1_n = w1.copy(deep=True)

covs1 = []
covs1_d = []
covs1_n = []
np.random.seed(0)
for i in range(nTrials):
    mu1, cov1, X = simCovMu(mu0, cov0, nObs, shrink = shrink)
    Xs.append(X)
    if minVarPortf: mu1 = None
    cov1_d = deNoiseCov(cov1, nObs*1./cov1.shape[1], bWidth)
    w1.loc[i] = minimize_portfolio_variance(cov1).flatten()   #optPort(cov1, mu1).flatten() # add column vector w as row in w1
    w1_d.loc[i] = minimize_portfolio_variance(cov1_d).flatten() #optPort(cov1_d, mu1).flatten() # np.sum(w1_d, axis=1) is vector of 1's. sum(np.sum(w1_d, axis=0)= nTrials
    # so minimum-variance-portfolio is 1./nTrials*(np.sum(w1_d, axis=0)) - but distribution not stationary
   # tensorX = X_to_tensor(X, lookback)
   # weights_n = network(tensorX)
   # w1_n.loc[i] = weights_n.detach().numpy().flatten()#network(tensorX)
    
    
    covs1.append(cov1)
    covs1_d.append(cov1_d)
    
min_var_port = 1./nTrials*(np.sum(w1_d, axis=0)) 
#code snippet 2.11
w0 = optPort(cov0, None if minVarPortf else mu0) # w0 true percentage asset allocation
w0 = np.repeat(w0.T, w1.shape[0], axis=0) 
rmsd = np.mean((w1-w0).values.flatten()**2)**.5     #RMSE not denoised
rmsd_d = np.mean((w1_d-w0).values.flatten()**2)**.5 #RMSE denoised
#rmsd_n = np.mean((w1_n-w0).values.flatten()**2)**.5 #
print("RMSE not denoised:"+str( rmsd))
print("RMSE denoised:"+str( rmsd_d))
#print("RMSE neural:"+str( rmsd_n))

sse:13.741536354262282
sse:13.741518536759038
sse:1313.6945493704338
sse:1313.6945672618524
sse:204.21091607191744
sse:204.21100637038586
sse:10.86118739887756
sse:10.861187764550628
sse:10.859911642078707
sse:10.859911655869926
sse:10.859909833668711
sse:10.859909833656046
sse:10.859909833660943
sse:10.85990983366076
found errPDFs0.5032732013036696
sse:15.256928137631423
sse:15.256909533378346
sse:1322.0852929847624
sse:1322.085312847858
sse:213.42495383260962
sse:213.42504637149074
sse:12.232937273728886
sse:12.232938063670996
sse:12.227198154348496
sse:12.227198183658803
sse:12.227190277294508
sse:12.227190277234152
sse:12.227190277231056
sse:12.227190277231042
found errPDFs0.5032925086262803
sse:14.511293079332502
sse:14.511272786016187
sse:1310.3371946291227
sse:1310.3372148367218
sse:235.6724870454185
sse:235.67258009136802
sse:11.022575138516135
sse:11.022576308452921
sse:11.010231982742162
sse:11.010232034037426
sse:11.010208336675774
sse:11.010208336502107
sse:11.0102083363179

## Robust

In [18]:
# network.load_state_dict(torch.load('bachelier_with_shorting_250assets_min_var_loss.pth'))


np.random.seed(32)
# block is a sector with bSize assets
nBlocks, bSize, bCorr =10, 50, .5
np.random.seed(0)
mu0, cov0 = formTrueMatrix(nBlocks, bSize, bCorr)
Xs = []

nObs, nTrials, bWidth, shrink, minVarPortf = 10000, 15, .01, False, True
w1 = pd.DataFrame(columns = range(cov0.shape[0]), index = range(nTrials), dtype=float)
w1_d = w1.copy(deep=True)
# w1_n = w1.copy(deep=True)

covs1 = []
covs1_d = []
covs1_n = []
np.random.seed(0)
for i in range(nTrials):
    mu1, cov1, X = simCovMuRob(mu0, cov0, nObs)
    Xs.append(X)
    if minVarPortf: mu1 = None
    cov1_d = deNoiseCov(cov1, nObs*1./cov1.shape[1], bWidth)
    w1.loc[i] = minimize_portfolio_variance(cov1).flatten()   #optPort(cov1, mu1).flatten() # add column vector w as row in w1
    w1_d.loc[i] = minimize_portfolio_variance(cov1_d).flatten() #optPort(cov1_d, mu1).flatten() # np.sum(w1_d, axis=1) is vector of 1's. sum(np.sum(w1_d, axis=0)= nTrials
    # so minimum-variance-portfolio is 1./nTrials*(np.sum(w1_d, axis=0)) - but distribution not stationary
   # tensorX = X_to_tensor(X, lookback)
   # weights_n = network(tensorX)
   # w1_n.loc[i] = weights_n.detach().numpy().flatten()#network(tensorX)
    
    
    covs1.append(cov1)
    covs1_d.append(cov1_d)
    
min_var_port = 1./nTrials*(np.sum(w1_d, axis=0)) 
#code snippet 2.11
w0 = optPort(cov0, None if minVarPortf else mu0) # w0 true percentage asset allocation
w0 = np.repeat(w0.T, w1.shape[0], axis=0) 
rmsd = np.mean((w1-w0).values.flatten()**2)**.5     #RMSE not denoised
rmsd_d = np.mean((w1_d-w0).values.flatten()**2)**.5 #RMSE denoised
#rmsd_n = np.mean((w1_n-w0).values.flatten()**2)**.5 #
print("RMSE not denoised:"+str( rmsd))
print("RMSE denoised:"+str( rmsd_d))
#print("RMSE neural:"+str( rmsd_n))

C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13892.404992281628438 > -13903.054419875321400). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13886.843039208200935 > -13921.472062477332656). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13904.545688698923186 > -13912.069251621640433). You may want to try with

C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13886.559653818245351 > -13905.368536950838461). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13890.789802620205592 > -13900.884651315893279). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13895.714039277041593 > -13898.999046834891487). You may want to try with

C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13884.150603466538087 > -13885.992820745348581). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13881.686611484379682 > -13900.453203657199992). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13897.513550011097323 > -13898.486159839927495). You may want to try with

C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13901.629946075858243 > -13904.116233911850941). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13885.059867087449675 > -13923.755793295431431). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13882.661022883252372 > -13904.805044905924660). You may want to try with

sse:135.9296821199424
sse:135.9296752806249
sse:1200.7188926012577
sse:1200.7189037868866
sse:141.07344612787742
sse:141.07345797776048
sse:133.3553185357907
sse:133.3553184100953
sse:133.35452507868135
sse:133.35452508786085
sse:133.3545208354181
sse:133.35452083540883
sse:133.354520835409
sse:133.3545208354093
found errPDFs0.507397784153335


...

sse:124.97166109577411
sse:124.97165057215153
sse:1199.1363997902172
sse:1199.136411577791
sse:139.786548027268
sse:139.78657091851065
sse:119.7723335714578
sse:119.77233318475572
sse:119.7649490825809
sse:119.76494908076805
sse:119.76494892111131
sse:119.764948921126
found errPDFs0.5103188140803544


C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13904.209453463656246 > -13916.696013625369233). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13880.063690388931718 > -13882.920081638680131). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13869.759504128747722 > -13888.140401994382046). You may want to try with

C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13895.419798095639635 > -13904.249977679231961). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13904.609177920890943 > -13916.575420875009513). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13893.412109928211066 > -13905.599919619537104). You may want to try with

C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13904.769945725867728 > -13920.702570991159519). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13905.946166823301610 > -13916.285640072559545). You may want to try with a higher value of support_fraction (current value: 0.528).
  warnings.warn(
C:\Users\r0913246\AppData\Roaming\Python\Python310\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-13893.694512078005573 > -13912.483346784905734). You may want to try with

KeyboardInterrupt: 

# Check perfomance

In [110]:
q = nObs / (nBlocks * bSize)

In [111]:
corr11 = cov2corr(covs1[0])
eVal11, eVec11 = getPCA(corr11)
eMax11, var11 = findMaxEval(np.diag(eVal11), q, bWidth=.01)
nFacts11 = eVal11.shape[0]-np.diag(eVal11)[::-1].searchsorted(eMax11)


var11, nFacts11

sse:10.399669066372272
sse:10.399671894786536
sse:3022527575253.238
sse:3016491575610.4414
sse:473.7034319371673
sse:473.70335144392783
sse:8.312277393585834
sse:8.312277491688489
sse:8.310474598291874
sse:8.310474574806502
sse:8.310366245957992
sse:8.310366246113231
sse:8.310366241292005
sse:8.310366241292225
found errPDFs0.4862218327392375


(0.4862218327392375, 5)

In [112]:
corr11_d = cov2corr(covs1_d[5])
eVal11_d, eVec11_d = getPCA(corr11_d)
eMax11_d, var11_d = findMaxEval(np.diag(eVal11_d), q, bWidth=.01)
nFacts11_d = eVal11_d.shape[0]-np.diag(eVal11_d)[::-1].searchsorted(eMax11_d)


var11_d, nFacts11_d

sse:18345.153042029204
sse:18345.152664320565
sse:9260.121077081712
sse:9260.121165532631
found errPDFs0.99999


(0.99999, 5)

# NN

In [113]:
# num_subarrays = 10
# subarray_size = len(X) // num_subarrays
# subarrays = np.array([X[i * subarray_size:(i + 1) * subarray_size] for i in range(num_subarrays)])
 

# subarrays.shape

(10, 100, 250)

In [90]:
# def get_activation(name):
#     # This function will return a hook function that stores the output in a dictionary
#     def hook(model, input, output):
#         activations[name] = output.detach()
#     return hook

In [91]:
# inputs_to_allocation_layer = []

# for i in range(lookback, 1000):
#     # Attach the hook to the covariance_layer, which precedes the channel_collapse_layer
#     activations = {}
#     hook = network.portfolio_opt_layer.register_forward_hook(get_activation('portfolio_opt_layer'))

#     # Now run your data through the network. This will store the output of the covariance_layer in activations
#     test_X = X_to_tensor(X, lookback, i)
   
#     network(test_X)

#     # The output you're interested in is now stored in activations['covariance_layer_output']
#     input_to_allocation_layer = activations['portfolio_opt_layer']
#     inputs_to_allocation_layer.append(input_to_allocation_layer)
    
#     # Don't forget to remove the hook when you're done to prevent memory leaks
#     hook.remove()
    
    
# inputs_to_allocation_layer = np.array(inputs_to_allocation_layer)

In [92]:
# reshaped_inputs = inputs_to_allocation_layer.reshape(960, 250)
# inputs_to_allocation_layer[0, 0, 0], reshaped_inputs[0, 0]

NameError: name 'inputs_to_allocation_layer' is not defined

In [ ]:
# q = reshaped_inputs.shape[0]/reshaped_inputs.shape[1]

## Getting cov

In [114]:
test_X = X_to_tensor(X, lookback)

C:\Users\r0913246\AppData\Local\Temp\ipykernel_4108\2520597956.py:238: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(X_tensor, dtype=torch.float32)


In [115]:
test_X.shape

torch.Size([1, 1, 40, 250])

In [116]:
cov_n = network.get_covmat(test_X)[0].detach().numpy()

In [104]:
# cov_n = np.random.normal(size=(4000, 250))
# cov_n = np.cov(cov_n)

In [120]:
q = 100

In [121]:
corr11_n = cov2corr(cov_n)
eVal11_n, eVec11_n = getPCA(corr11_n)
eMax11_n, var11_n = findMaxEval(np.diag(eVal11_n), 100, bWidth=.01)
nFacts11_n = eVal11.shape[0]-np.diag(eVal11_n)[::-1].searchsorted(eMax11_n)
nFacts11_n, var11_n

sse:79440.96512236427
sse:79440.96317654636
sse:6580.109209273829
sse:6580.109331054971
found errPDFs0.99999


(19, 0.99999)

(45, 0.99999)